In [1]:
from data_utils import mark_buy_sell, get_formatted_csv

df = get_formatted_csv('DLINKINDIA.csv')
# df = mark_buy_sell_continous(df)
df = mark_buy_sell(df)

df.head()

  0%|          | 0/1138 [00:00<?, ?it/s]/home/parashar/anaconda3/envs/py310/lib/python3.10/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/parashar/anaconda3/envs/py310/lib/python3.10/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/parashar/anaconda3/envs/py310/lib/python3.10/site-packages/pandas/core/indexing.py:1817: Setting

,open,high,low,close,volume,action
datetime,,,,,,
2019-01-01 09:15:00,73.4,73.45,72.55,73.1,11022,hold
2019-01-01 10:15:00,73.1,73.50,72.10,73.3,2212,strong_sell
2019-01-01 11:15:00,73.3,73.30,73.00,73.0,513,hold
2019-01-01 12:15:00,73.0,73.00,72.10,72.8,567,strong_buy
2019-01-01 13:15:00,72.8,73.20,72.70,73.2,311,hold


# [MACD Indicator](https://www.investopedia.com/terms/m/macd.asp)


## Investors use the MACD to: 
* Traders use MACD to identify changes in the direction or strength of a stock’s price trend.


## MACD signals: 
* crossover

In [11]:
import pandas_ta as ta

df.ta.macd(append=True)
"""
Returns:
    pd.DataFrame: macd, histogram, signal columns.
"""

df.tail()

,open,high,low,close,volume,action,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
datetime,,,,,,,,,
2023-08-02 11:15:00,319.65,320.90,316.20,319.35,70920,hold,4.586115,1.444638,3.141477
2023-08-02 12:15:00,319.35,321.95,318.25,319.95,33586,hold,4.533217,1.113393,3.419825
2023-08-02 13:15:00,319.95,320.00,313.00,313.05,85282,hold,3.889686,0.375889,3.513797
2023-08-02 14:15:00,313.05,319.50,311.00,318.75,123367,strong_buy,3.795868,0.225657,3.570211
2023-08-02 15:15:00,318.75,318.75,312.80,317.00,50727,hold,3.539505,-0.024565,3.564070


### Plot labelled Data

In [14]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm
import plotly.express as px

_df = df[df['action'] != 'hold']
# _df=_df.head(100)

# Create the multiline chart using Plotly
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)


# Add traces for high, low, BBU, and BBL
fig.add_trace(go.Scatter( x=_df.index, y=_df['open'], mode='lines', name='Open'))
fig.add_trace(go.Scatter( x=_df.index, y=_df['high'], mode='lines', name='High'))
fig.add_trace(go.Scatter( x=_df.index, y=_df['low'], mode='lines', name='Low'))
fig.add_trace(go.Scatter( x=_df.index, y=_df['close'], mode='lines', name='Close'))

fig.add_trace(go.Scatter(x=_df.index, y=_df['MACD_12_26_9'], mode='lines+lines', name='MACD fast'),row=2,col=1)
fig.add_trace(go.Scatter(x=_df.index, y=_df['MACDs_12_26_9'], mode='lines+lines', name='MACD slow'),row=2,col=1)

# TODO: figure out histogram
# fig.add_trace(go.Histogram(x=_df.index, y=_df['MACDh_12_26_9'],name='MACD histogram', autobinx=False),row=2,col=1)


sell_df = _df[_df['action'] == 'strong_sell']
fig.add_trace(go.Scatter(x=sell_df.index, y=sell_df['high'],
                mode='markers',
                marker=dict(color='red', symbol='star-triangle-down'),
                name='Sell',
                showlegend=True ))

buy_df = _df[_df['action'] == 'strong_buy']
fig.add_trace(go.Scatter(x=buy_df.index, y=buy_df['high'],
            mode='markers',
            marker=dict(color='green', symbol='star-triangle-up'),
            name='Buy',
            showlegend=True ))


# Update layout
fig.update_layout(
    xaxis=dict(title='Date'),
    yaxis=dict(title='Value'),
    title='Multiline Chart with Datetime X-axis',
    showlegend=True,
    legend=dict(x=0, y=1),
    xaxis_tickangle=-45
)


# Show the Plotly figure
fig.show()